In [721]:
# pip install --upgrade spacy

In [722]:
# !python -m spacy download en_core_web_md

In [723]:
# pip install podium-nlp

In [724]:
# pip install hdbscan

In [725]:
# pip install bertopic

In [726]:
# pip install emoji

In [727]:
# pip install wordsegment

In [728]:
# pip install ekphrasis -U

In [729]:
# pip install contextualSpellCheck

In [1]:
from tqdm import tqdm
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from bertopic import BERTopic
from podium import Vocab, Field, LabelField
from podium.datasets import TabularDataset
from podium.vectorizers import GloVe
from nltk.sentiment import SentimentIntensityAnalyzer
from copy import deepcopy
from nltk.tokenize.casual import TweetTokenizer
from time import time
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from ekphrasis.classes.segmenter import Segmenter

import contextualSpellCheck
import nltk
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
import re
import emoji
import warnings
warnings.filterwarnings("ignore")

In [731]:
# from google.colab import drive
# drive.mount('/content/drive')

In [732]:
# %cd drive/MyDrive/datasets/

# Preprocessing

In [113]:
nlp = spacy.load("en_core_web_md")

In [114]:
df = pd.read_csv("train/SemEval2018-T3-train-taskA_emoji.txt", sep='\t', lineterminator='\n', encoding='utf-8')
df_test = pd.read_csv("goldtest_TaskA/SemEval2018-T3_gold_test_taskA_emoji.txt", sep='\t', lineterminator='\n', encoding='utf-8')
df_replace = pd.read_csv("test_TaskA/SemEval2018-T3_input_test_taskA.txt", sep='\t', lineterminator='\n', encoding='utf-8')

In [115]:
df.rename({'Tweet text\r': 'Tweet text'}, inplace=True, axis=1)
df_test.rename({'Tweet text\r': 'Tweet text'}, inplace=True, axis=1)
df_replace.rename({'Tweet text\r': 'Tweet text'}, inplace=True, axis=1)

## Getting clean text

In [116]:
tokenizer = TweetTokenizer(preserve_case=True, reduce_len=True, strip_handles=True)
df_tmp = df['Tweet text'].apply(tokenizer.tokenize)
df_test_tmp = df['Tweet text'].apply(tokenizer.tokenize)

In [6]:
# df['Tweet text'] = df['Tweet text'].apply(emoji.demojize)
# df_test['Tweet text'] = df_test['Tweet text'].apply(emoji.demojize)

In [117]:
def emoji_counter(s):
    return len(emoji.emoji_lis(s))

df['emoji_count'] = df['Tweet text'].apply(emoji_counter)

In [739]:
# def filter_emojis(s):
#     print(f'S: {s}')
#     print()
#     d = emoji.emoji_lis(emoji.emojize(s))
#     s_new = ''
#     if d != []:
#         for i, el in enumerate(d):
#             print(i, el)
#             if i == 0:
#                 s_new += s[:el['location']]
#             elif i < len(d):
#                 up_to = sum([k['location'] + len(emoji.demojize(k['emoji'])) for k in d[:i]])
#                 len_text = el['location'] - (d[i-1]['location'] + 1)
#                 s_new += s[up_to : up_to + len_text]
#     return s

In [118]:
#seg_eng = Segmenter(corpus="english")
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
seg_tw = Segmenter(corpus="twitter")

Reading twitter - 1grams ...
Reading twitter - 2grams ...


In [119]:
def separate_hashtags(s):
    '''Removes the hashtag sign and segments the hashtag text.'''
    hashtags = []
    
    l = []
    for i, s_i in enumerate(s):
        if s_i.startswith('#'):
            tmp = tokenizer.tokenize(seg_tw.segment(s_i.replace('#', '')))
            l.extend(tmp)
        else:
            l.append(s_i)
    return l
            
df_tmp = df_tmp.apply(separate_hashtags)
df_test_tmp = df_test_tmp.apply(separate_hashtags)

In [121]:
def preprocess_words(s):
    '''
    Removes tags, emojis, links, smiley faces, | signs, stopwords and changes the case to lower.
    '''
    ret_list = []

    smiley_regex = r'([\:\;\=][()PDO\/\]\[p|]+)+'
    
    is_tag = lambda w: w.startswith('@')
    is_vertical_line = lambda w: w.startswith('|')
    is_emoji = lambda w: emoji.is_emoji(w)
    # is_emoji = lambda w: w != ':' and w.startswith(':') and w.endswith(':')
    # remove_emoji = lambda w: w[:w.index(':')] + w[w.rindex(':')+1:] if ':' in w and w.index(':') != w.rindex(':') else w
    is_link = lambda w: w.startswith("http") or w.startswith("https")
    is_hashtag = lambda w: w.startswith("#")
    is_smiley = lambda w: re.match(smiley_regex, w)

    w2 = []
    for i, w in enumerate(s):
        if is_tag(w) or is_emoji(w) or is_link(w) or is_vertical_line(w):
            continue

        elif is_hashtag(w):
            w_tmp = w.replace('#', '')
            if w_tmp != '':
                lower_append(w_tmp, w2)

        elif is_smiley(w):
            w_tmp = re.sub(smiley_regex, '', w)
            if w_tmp != '':
                lower_append(w_tmp, w2)

        else:
            w_tmp = w.replace('#', '')
            w_tmp = w_tmp.replace('|', '')
            w_tmp = w_tmp.replace('_', '')
            w_tmp = w_tmp.replace('...', '')
            if w_tmp != '':
                lower_append(w_tmp, w2)

    return ' '.join([i for i in w2 if len(i) > 2])

def lower_append(w, l):
    l.append(w.lower())

df['clean_text'] = df_tmp.apply(preprocess_words)
df_test['clean_text'] = df_test_tmp.apply(preprocess_words)

In [122]:
print(df.shape)
df.head(15)

(3817, 5)


,Tweet index,Label,Tweet text,emoji_count,clean_text
0,1,1,Sweet United Nations video. Just in time for C...,0,sweet united nations video just time for chris...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,0,are rumored have talked erv's agent and the an...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,0,hey there nice see you minnesota winter weather
3,4,0,3 episodes left I'm dying over here\r,0,episodes left i'm dying over here
4,5,1,I can't breathe! was chosen as the most notabl...,0,can't breathe was chosen the most notable quot...
5,6,0,You're never too old for Footie Pajamas. http:...,0,you're never too old for footie pajamas
6,7,1,Nothing makes me happier then getting on the h...,0,nothing makes happier then getting the highway...
7,8,0,4:30 an opening my first beer now gonna be a l...,0,4:30 opening first beer now gonna long night day
8,9,0,@Adam_Klug do you think you would support a gu...,0,you think you would support guy who knocked ou...
9,10,0,@samcguigan544 You are not allowed to open tha...,0,you are not allowed open that until christmas day


In [ ]:
# def simple_preprocessing(s):
#     '''Lowercases, tokenizes, de-accents, removes words shorter than 3 and longer than 14 characters'''
#     return [' '.join(simple_preprocess(s_i)) for s_i in s]

# df['clean_text'] = df[['clean_text']].apply(simple_preprocessing)
# df_test['clean_text'] = df_test[['clean_text']].apply(simple_preprocessing)

In [123]:
print(df.shape)
df.head(15)

(3817, 5)


,Tweet index,Label,Tweet text,emoji_count,clean_text
0,1,1,Sweet United Nations video. Just in time for C...,0,sweet united nations video just time for chris...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,0,are rumored have talked erv's agent and the an...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,0,hey there nice see you minnesota winter weather
3,4,0,3 episodes left I'm dying over here\r,0,episodes left i'm dying over here
4,5,1,I can't breathe! was chosen as the most notabl...,0,can't breathe was chosen the most notable quot...
5,6,0,You're never too old for Footie Pajamas. http:...,0,you're never too old for footie pajamas
6,7,1,Nothing makes me happier then getting on the h...,0,nothing makes happier then getting the highway...
7,8,0,4:30 an opening my first beer now gonna be a l...,0,4:30 opening first beer now gonna long night day
8,9,0,@Adam_Klug do you think you would support a gu...,0,you think you would support guy who knocked ou...
9,10,0,@samcguigan544 You are not allowed to open tha...,0,you are not allowed open that until christmas day


In [124]:
def remove_tuple_characters(s):
    return [re.sub(r'(.)\1{2,}', r'\1', w) for w in s]

df['clean_text'] = df[['clean_text']].apply(remove_tuple_characters)
df_test['clean_text'] = df_test[['clean_text']].apply(remove_tuple_characters)

print(df.shape)
df.head(15)

(3817, 5)


,Tweet index,Label,Tweet text,emoji_count,clean_text
0,1,1,Sweet United Nations video. Just in time for C...,0,sweet united nations video just time for chris...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,0,are rumored have talked erv's agent and the an...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,0,hey there nice see you minnesota winter weather
3,4,0,3 episodes left I'm dying over here\r,0,episodes left i'm dying over here
4,5,1,I can't breathe! was chosen as the most notabl...,0,can't breathe was chosen the most notable quot...
5,6,0,You're never too old for Footie Pajamas. http:...,0,you're never too old for footie pajamas
6,7,1,Nothing makes me happier then getting on the h...,0,nothing makes happier then getting the highway...
7,8,0,4:30 an opening my first beer now gonna be a l...,0,4:30 opening first beer now gonna long night day
8,9,0,@Adam_Klug do you think you would support a gu...,0,you think you would support guy who knocked ou...
9,10,0,@samcguigan544 You are not allowed to open tha...,0,you are not allowed open that until christmas day


In [125]:
def lemmatize(s):
    '''Lemmatizes the words in the sentences and returns them if theyre not stopwords or punctuation'''
    return [[w.lemma_.lower() for w in nlp(s_i) if w.lemma_.lower() not in nlp.Defaults.stop_words and not w.is_punct] for s_i in s]

df['lemmas'] = df[['clean_text']].apply(lemmatize)
df_test['lemmas'] = df_test[['clean_text']].apply(lemmatize)

In [126]:
df.head(15)

,Tweet index,Label,Tweet text,emoji_count,clean_text,lemmas
0,1,1,Sweet United Nations video. Just in time for C...,0,sweet united nations video just time for chris...,"[sweet, united, nations, video, time, christma..."
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,0,are rumored have talked erv's agent and the an...,"[rumor, talk, erv, agent, angel, ask, escobar,..."
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,0,hey there nice see you minnesota winter weather,"[hey, nice, minnesota, winter, weather]"
3,4,0,3 episodes left I'm dying over here\r,0,episodes left i'm dying over here,"[episode, leave, die]"
4,5,1,I can't breathe! was chosen as the most notabl...,0,can't breathe was chosen the most notable quot...,"[breathe, choose, notable, quote, year, annual..."
5,6,0,You're never too old for Footie Pajamas. http:...,0,you're never too old for footie pajamas,"[old, footie, pajama]"
6,7,1,Nothing makes me happier then getting on the h...,0,nothing makes happier then getting the highway...,"[happy, highway, break, light, light, like, ch..."
7,8,0,4:30 an opening my first beer now gonna be a l...,0,4:30 opening first beer now gonna long night day,"[4:30, open, beer, going, long, night, day]"
8,9,0,@Adam_Klug do you think you would support a gu...,0,you think you would support guy who knocked ou...,"[think, support, guy, knock, daughter, rice, d..."
9,10,0,@samcguigan544 You are not allowed to open tha...,0,you are not allowed open that until christmas day,"[allow, open, christmas, day]"


In [127]:
### baseline features ###
tokenizer2 = TweetTokenizer()
def word_counter(s):
    return len([x for x in tokenizer.tokenize(s) if not x.startswith(("@", "#", "http"))])

def char_counter(s):
    return len(s.replace(' ', ''))

def all_uppercase_counter(s):
    return len([x for x in tokenizer2.tokenize(s) if x.isupper() and not x.startswith(("@", "#", "http"))])

def all_lowercase_counter(s):
    return len([x for x in tokenizer2.tokenize(s) if x.islower() and not x.startswith(("@", "#", "http"))])

def capitalised_counter(s):
    return sum([i.istitle() for i in tokenizer2.tokenize(s)])

def digit_counter(s):
    return sum([i.isdigit() for i in s])



### other features ###
def tag_counter(s):
    return len([x for x in tokenizer2.tokenize(s) if x.startswith("@")])

def hashtag_counter(s):
    return len([x for x in tokenizer2.tokenize(s) if x.startswith("#")])

def link_counter(s):
    return len([x for x in tokenizer2.tokenize(s) if x.startswith(('http:', 'https:'))])

def smiley_counter(s):
    return len([x for x in tokenizer2.tokenize(s) if re.match(r'([\:\;\=][()PDO\/\]\[p|]+)+', x)])

def exclamation_mark_counter(s):
    return s.count('!')

def question_mark_counter(s):
    return s.count('?')

def ellipsis_counter(s):
    return s.count('...')
    


### NER ###
def ORG_tag_counter(s):
    doc = nlp(s)
    return len([d.text for d in doc.ents if d.label_ == 'ORG'])

def NORP_tag_counter(s):
    doc = nlp(s)
    return len([d.text for d in doc.ents if d.label_ == 'NORP'])

def GPE_tag_counter(s):
    doc = nlp(s)
    return len([d.text for d in doc.ents if d.label_ == 'GPE'])

def PERSON_tag_counter(s):
    doc = nlp(s)
    return len([d.text for d in doc.ents if d.label_ == 'PERSON'])

In [128]:
def add_features(some_df):
    some_df['word_count'] = df['clean_text'].apply(word_counter)
    some_df['char_count'] = df['clean_text'].apply(char_counter)
    some_df['all_uppercase_count'] = df['Tweet text'].apply(all_uppercase_counter)
    some_df['all_lowercase_count'] = df['Tweet text'].apply(all_lowercase_counter)
    some_df['capitalised_count'] = df['Tweet text'].apply(capitalised_counter)
    some_df['digit_count'] = df['Tweet text'].apply(digit_counter)
    
    some_df['tag_count'] = df['Tweet text'].apply(tag_counter)
    some_df['hashtag_count'] = df['Tweet text'].apply(hashtag_counter)
    some_df['link_count'] = df['Tweet text'].apply(link_counter)
    some_df['smiley_count'] = df['Tweet text'].apply(smiley_counter)
    
    some_df['exclamation_mark_count'] = df['Tweet text'].apply(exclamation_mark_counter)
    some_df['question_mark_count'] = df['Tweet text'].apply(question_mark_counter)
    some_df['ellipsis_count'] = df['Tweet text'].apply(ellipsis_counter)
    
    some_df['ORG_tag_count'] = df['Tweet text'].apply(ORG_tag_counter)
    some_df['NORP_tag_count'] = df['Tweet text'].apply(NORP_tag_counter)
    some_df['GPE_tag_count'] = df['Tweet text'].apply(GPE_tag_counter)
    some_df['PERSON_tag_count'] = df['Tweet text'].apply(PERSON_tag_counter)
    
add_features(df)
add_features(df_test)

In [129]:
df.head()

,Tweet index,Label,Tweet text,emoji_count,clean_text,lemmas,word_count,char_count,all_uppercase_count,all_lowercase_count,...,hashtag_count,link_count,smiley_count,exclamation_mark_count,question_mark_count,ellipsis_count,ORG_tag_count,NORP_tag_count,GPE_tag_count,PERSON_tag_count
0,1,1,Sweet United Nations video. Just in time for C...,0,sweet united nations video just time for chris...,"[sweet, united, nations, video, time, christma...",10,58,0,4,...,2,1,0,0,0,0,1,0,0,0
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,0,are rumored have talked erv's agent and the an...,"[rumor, talk, erv, agent, angel, ask, escobar,...",15,78,0,14,...,0,0,1,0,0,2,1,0,0,2
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,0,hey there nice see you minnesota winter weather,"[hey, nice, minnesota, winter, weather]",8,40,1,4,...,0,0,0,1,0,0,0,0,1,0
3,4,0,3 episodes left I'm dying over here\r,0,episodes left i'm dying over here,"[episode, leave, die]",6,28,0,5,...,0,0,0,0,0,0,0,0,0,0
4,5,1,I can't breathe! was chosen as the most notabl...,0,can't breathe was chosen the most notable quot...,"[breathe, choose, notable, quote, year, annual...",16,88,1,20,...,0,0,0,1,0,0,1,0,0,0


In [130]:
df_train = df.sample(frac=0.7)
df_validation = df.drop(df_train.index)

In [131]:
df_train.head()

,Tweet index,Label,Tweet text,emoji_count,clean_text,lemmas,word_count,char_count,all_uppercase_count,all_lowercase_count,...,hashtag_count,link_count,smiley_count,exclamation_mark_count,question_mark_count,ellipsis_count,ORG_tag_count,NORP_tag_count,GPE_tag_count,PERSON_tag_count
373,374,0,@MrMindMiracle One of those obvious when you s...,0,one those obvious when you see but clever idea...,"[obvious, clever, idea, glad, approve]",12,54,0,14,...,0,0,0,3,0,0,0,0,0,0
2217,2221,1,It's a Christmas miracle!|| http://t.co/3X7cxk...,0,it's christmas miracle,"[christmas, miracle]",3,20,0,2,...,0,1,0,1,0,0,0,0,0,0
232,233,0,@badassbraeden Yeah no I'm still laughing. It'...,0,yeah i'm still laughing it's not even that fun...,"[yeah, laugh, funny, laugh]",13,59,0,10,...,0,0,0,0,0,0,0,0,1,0
2703,2707,0,http://t.co/8ICPDcAoNQ #CHRISTMAS #GIFT #BLESS...,0,christmas gift blessing santa follow the inter...,"[christmas, gift, blessing, santa, follow, int...",11,62,0,0,...,8,2,0,0,0,1,0,0,0,0
924,925,1,I picked a great week to start a new show on N...,0,picked great week start new show netflix hell ...,"[pick, great, week, start, new, netflix, hell,...",9,44,1,10,...,1,0,0,0,0,0,1,0,0,0


In [132]:
def new_df_with_all_features(df1, df2):
    cols_to_add = list(set(df2.columns.tolist()) - set(df1.columns.tolist()))
    if 'Tweet index' in cols_to_add:
        cols_to_add.remove('Tweet index')
    new_df = pd.concat((df1.copy(), df2[cols_to_add]), axis=1)
    return new_df

In [133]:
def join_docs(s):
    '''Joins the strings inside the inner list of a nested list'''
    return ' '.join(s)

df_train['topic_text'] = df_train['lemmas'].apply(join_docs)
df_validation['topic_text'] = df_validation['lemmas'].apply(join_docs)
df_test['topic_text'] = df_test['lemmas'].apply(join_docs)

In [134]:
df_train.shape

(2672, 24)

In [135]:
df_train.head()

,Tweet index,Label,Tweet text,emoji_count,clean_text,lemmas,word_count,char_count,all_uppercase_count,all_lowercase_count,...,link_count,smiley_count,exclamation_mark_count,question_mark_count,ellipsis_count,ORG_tag_count,NORP_tag_count,GPE_tag_count,PERSON_tag_count,topic_text
373,374,0,@MrMindMiracle One of those obvious when you s...,0,one those obvious when you see but clever idea...,"[obvious, clever, idea, glad, approve]",12,54,0,14,...,0,0,3,0,0,0,0,0,0,obvious clever idea glad approve
2217,2221,1,It's a Christmas miracle!|| http://t.co/3X7cxk...,0,it's christmas miracle,"[christmas, miracle]",3,20,0,2,...,1,0,1,0,0,0,0,0,0,christmas miracle
232,233,0,@badassbraeden Yeah no I'm still laughing. It'...,0,yeah i'm still laughing it's not even that fun...,"[yeah, laugh, funny, laugh]",13,59,0,10,...,0,0,0,0,0,0,0,1,0,yeah laugh funny laugh
2703,2707,0,http://t.co/8ICPDcAoNQ #CHRISTMAS #GIFT #BLESS...,0,christmas gift blessing santa follow the inter...,"[christmas, gift, blessing, santa, follow, int...",11,62,0,0,...,2,0,0,0,1,0,0,0,0,christmas gift blessing santa follow interview...
924,925,1,I picked a great week to start a new show on N...,0,picked great week start new show netflix hell ...,"[pick, great, week, start, new, netflix, hell,...",9,44,1,10,...,0,0,0,0,0,1,0,0,0,pick great week start new netflix hell wheel


## Topic modeling baselines

In [ ]:
count_vectorizer = CountVectorizer(
    analyzer='word',
    min_df=20,
    max_df=0.5,
)

tfidf_vectorizer = TfidfVectorizer(
    analyzer='word',
    min_df=20,
    max_df=0.5,
)

tweet_text_count_train = count_vectorizer.fit_transform(df_train['topic_text'])
tweet_text_count_validation = count_vectorizer.transform(df_validation['topic_text'])
tweet_text_count_test = count_vectorizer.transform(df_test['topic_text'])

tweet_text_tfidf_train = tfidf_vectorizer.fit_transform(df_train['topic_text'])
tweet_text_tfidf_validation = tfidf_vectorizer.transform(df_validation['topic_text'])
tweet_text_tfidf_test = tfidf_vectorizer.transform(df_test['topic_text'])

In [ ]:
inertia = []
range_ = list(range(2, 60))
for i in range_:
    model = KMeans(i)
    model.fit(tweet_text_count_train)
    inertia.append(model.inertia_)
    
plt.plot(range_, inertia)
plt.xlabel('Number of components')
plt.ylabel('Inertia')
plt.show()

In [ ]:
model = KMeans(17)
model.fit(tweet_text_count_train)

kmeans_count_labels_train = model.predict(tweet_text_count_train)
kmeans_count_labels_validation = model.predict(tweet_text_count_validation)
kmeans_count_labels_test = model.predict(tweet_text_count_test)

In [ ]:
inertia = []
for i in range_:
    model = KMeans(i)
    model.fit(tweet_text_tfidf_train)
    inertia.append(model.inertia_)
plt.plot(range_, inertia)
plt.xlabel('Number of components')
plt.ylabel('Inertia')
plt.show()

In [ ]:
model = KMeans(22)
model.fit(tweet_text_tfidf_train)

kmeans_tfidf_labels_train = model.predict(tweet_text_tfidf_train)
kmeans_tfidf_labels_validation = model.predict(tweet_text_tfidf_validation)
kmeans_tfidf_labels_test = model.predict(tweet_text_tfidf_test)

### BERTopic

In [136]:
df_train.reset_index(drop=True, inplace=True)
df_validation.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

In [153]:
start = time()
topic_model = BERTopic(top_n_words=8, min_topic_size=20, calculate_probabilities=True)
topics, probs = topic_model.fit_transform(df_train['topic_text'])
end = time()
print(end-start)
pred_train = topic_model.transform(df_train['topic_text'])
pred_validation = topic_model.transform(df_validation['topic_text'])
pred_test = topic_model.transform(df_test['topic_text'])

22.638654708862305


In [154]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1158,-1_day_work_love_like
1,0,210,0_funny_think_lol_know
2,1,203,1_black_police_obama_race
3,2,154,2_team_game_playoff_win
4,3,145,3_look_girl_sock_wear
5,4,111,4_christmas_gift_xmas_merry
6,5,109,5_sleep_morning_wake_hour
7,6,94,6_study_class_school_teacher
8,7,87,7_twitter_tweet_follow_retweet
9,8,67,8_music_photo_listen_drink


In [155]:
probability_threshold = 0.07
new_topics = [np.argmax(prob) if max(prob) >= probability_threshold else -1 for prob in pred_train[1]]
np.unique(new_topics, return_counts=True)

(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16], dtype=int64),
 array([746, 233, 243, 146, 275,  99, 102, 111,  84,  72, 111,  77,  70,
         77,  50,  38,  58,  80], dtype=int64))

In [156]:
probability_threshold = 0.07
new_topics_train = [np.argmax(prob) if max(prob) >= probability_threshold else -1 for prob in pred_train[1]]
new_topics_validation = [np.argmax(prob) if max(prob) >= probability_threshold else -1 for prob in pred_validation[1]]
new_topics_test = [np.argmax(prob) if max(prob) >= probability_threshold else -1 for prob in pred_test[1]]
np.unique(new_topics_train, return_counts=True)

(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16], dtype=int64),
 array([746, 233, 243, 146, 275,  99, 102, 111,  84,  72, 111,  77,  70,
         77,  50,  38,  58,  80], dtype=int64))

In [ ]:
# df_train_bertopic = df_train.copy()
# df_validation_bertopic = df_validation.copy()
# df_test_bertopic = df_test.copy()

In [159]:
df_train['topic'] = new_topics_train
df_validation['topic'] = new_topics_validation
df_test['topic'] = new_topics_test

In [ ]:
# df_train_bertopic.head()

## Embedding

In [160]:
tweet_embedding_train = df_train[['topic_text', 'Label']]
tweet_embedding_validation = df_validation[['topic_text', 'Label']]
tweet_embedding_test = df_test[['topic_text', 'Label']]

In [161]:
tweet_embedding_train.reset_index(drop=True, inplace=True)
tweet_embedding_validation.reset_index(drop=True, inplace=True)
tweet_embedding_test.reset_index(drop=True, inplace=True)

In [162]:
max_vocab_size = 10_000
vocab = Vocab(max_size=max_vocab_size, min_freq=20)

TWEET = Field('text', numericalizer=vocab)
LABEL = LabelField('Label')

fields = [TWEET, LABEL]

train = TabularDataset.from_pandas(df_train[['topic_text', 'Label']], fields)
validation = TabularDataset.from_pandas(df_validation[['topic_text', 'Label']], fields)
test = TabularDataset.from_pandas(df_test[['topic_text', 'Label']], fields)
train.finalize_fields()

glove = GloVe()
embeddings = glove.load_vocab(vocab)

train_batch = train.batch(add_padding=True)
validation_batch = validation.batch(add_padding=True)
test_batch = test.batch(add_padding=True)

In [163]:
train_batch['text'].astype(int)[0:5]

array([[ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1],
       [10,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1],
       [45,  0, 67,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1

In [164]:
tweet_train = embeddings[train_batch['text'].astype(int)]
tweet_validation = embeddings[validation_batch['text'].astype(int)]
tweet_test = embeddings[test_batch['text'].astype(int)]

# Mean
tweet_train_mean = tweet_train.mean(axis=1)
tweet_validation_mean = tweet_validation.mean(axis=1)
tweet_test_mean = tweet_test.mean(axis=1)

In [165]:
embeddings_train_mean_df = pd.DataFrame(tweet_train_mean)
df_train = pd.merge(df_train, embeddings_train_mean_df, left_index=True, right_index=True)

embeddings_validation_mean_df = pd.DataFrame(tweet_validation_mean)
df_validation = pd.merge(df_validation, embeddings_validation_mean_df, left_index=True, right_index=True)

embeddings_test_mean_df = pd.DataFrame(tweet_test_mean)
df_test = pd.merge(df_test, embeddings_test_mean_df, left_index=True, right_index=True)

In [38]:
df_train.head()

,Tweet index,Label,Tweet text,emoji_count,clean_text,lemmas,word_count,char_count,all_uppercase_count,all_lowercase_count,...,290,291,292,293,294,295,296,297,298,299
0,2484,1,Babysitting is what I like to do on a Saturday...,1,babysitting what like saturday unamusedface,"[babysitte, like, saturday, unamusedface]",5,39,1,8,...,-0.374424,-3.264361,0.275496,0.532334,-0.278861,-0.158356,1.689196,-1.279314,-0.930186,2.086180
1,2494,0,@BigAlMLM Please give me a Follow Back.\r,0,please give follow back,[follow],4,20,0,3,...,-0.436824,-3.424057,0.319748,0.563718,-0.227369,-0.133755,1.794007,-1.279203,-1.040222,2.162534
2,1297,1,Obama wants closer relations with a Marxist na...,0,obama wants closer relations with marxist nati...,"[obama, want, close, relation, marxist, nation...",13,70,3,8,...,-0.263332,-2.963807,0.208874,0.466685,-0.358867,-0.203663,1.484307,-1.259718,-0.724626,1.928348
3,3755,1,Reading the Dundee United chairmanship stateme...,0,reading the dundee united chairmanship stateme...,"[read, dundee, united, chairmanship, statement...",18,96,0,19,...,-0.251506,-2.910116,0.199453,0.470484,-0.359425,-0.199702,1.467706,-1.243554,-0.719899,1.895914
4,3119,1,There's nothing like almost rear-ending someon...,0,there's nothing like almost rear-ending someon...,"[like, rear, end, slam, brake, reason, alert, ...",19,98,0,22,...,-0.290822,-3.009742,0.214149,0.487589,-0.339617,-0.174653,1.521694,-1.268621,-0.764868,1.950810


In [782]:
## Features
## Broj neg rijeci
## Broj poz rijeci
## Omjer
## Udaljenost izmedu poz i neg rijeci

In [39]:
nltk.download('vader_lexicon') # if error run this

[nltk_data] Downloading package vader_lexicon to C:\Users\Marino
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [40]:
def pos_neg_words(df, limit):
    sid = SentimentIntensityAnalyzer()

    neg_words = []
    neg_word_count = []
    pos_words = []
    pos_word_count = []
    for index, row in df.iterrows():
        lemmas = []
        if len(row['topic_text']) > 0:
            doc = nlp(row['topic_text'])
            for token in doc:
                lemmas.append(token.lemma_)

            current_pos = []
            current_neut = []
            current_neg = []
            for word in lemmas:
                if (sid.polarity_scores(word)['compound']) >= limit:
                    current_pos.append(word)
                elif (sid.polarity_scores(word)['compound']) <= -limit:
                    current_neg.append(word)
                else:
                    current_neut.append(word)

            neg_words.append(deepcopy(current_neg))
            neg_word_count.append(deepcopy(len(current_neg)))
            pos_words.append(deepcopy(current_pos))
            pos_word_count.append(deepcopy(len(current_pos)))
        else:
            neg_words.append([])
            neg_word_count.append(0)
            pos_words.append([])
            pos_word_count.append(0)
    return neg_words, neg_word_count, pos_words, pos_word_count

In [41]:
neg_words_train, neg_word_count_train, pos_words_train, pos_word_count_train = pos_neg_words(df_train, 0.2)
neg_words_val, neg_word_count_val, pos_words_val, pos_word_count_val = pos_neg_words(df_validation, 0.2)
neg_words_test, neg_word_count_test, pos_words_test, pos_word_count_test = pos_neg_words(df_test, 0.2)

In [42]:
df_train['neg_word_count'] = neg_word_count_train
df_train['pos_word'] = pos_words_train
df_train['pos_word_count'] = pos_word_count_train
df_train['neg_word'] = neg_words_train

df_validation['neg_word_count'] = neg_word_count_val
df_validation['pos_word'] = pos_words_val
df_validation['pos_word_count'] = pos_word_count_val
df_validation['neg_word'] = neg_words_val

df_test['neg_word_count'] = neg_word_count_test
df_test['pos_word'] = pos_words_test
df_test['pos_word_count'] = pos_word_count_test
df_test['neg_word'] = neg_words_test

df_train.head()

,Tweet index,Label,Tweet text,emoji_count,clean_text,lemmas,word_count,char_count,all_uppercase_count,all_lowercase_count,...,294,295,296,297,298,299,neg_word_count,pos_word,pos_word_count,neg_word
0,2484,1,Babysitting is what I like to do on a Saturday...,1,babysitting what like saturday unamusedface,"[babysitte, like, saturday, unamusedface]",5,39,1,8,...,-0.278861,-0.158356,1.689196,-1.279314,-0.930186,2.086180,0,[like],1,[]
1,2494,0,@BigAlMLM Please give me a Follow Back.\r,0,please give follow back,[follow],4,20,0,3,...,-0.227369,-0.133755,1.794007,-1.279203,-1.040222,2.162534,0,[],0,[]
2,1297,1,Obama wants closer relations with a Marxist na...,0,obama wants closer relations with marxist nati...,"[obama, want, close, relation, marxist, nation...",13,70,3,8,...,-0.358867,-0.203663,1.484307,-1.259718,-0.724626,1.928348,2,[],0,"[shock, shock]"
3,3755,1,Reading the Dundee United chairmanship stateme...,0,reading the dundee united chairmanship stateme...,"[read, dundee, united, chairmanship, statement...",18,96,0,19,...,-0.359425,-0.199702,1.467706,-1.243554,-0.719899,1.895914,0,"[united, care]",2,[]
4,3119,1,There's nothing like almost rear-ending someon...,0,there's nothing like almost rear-ending someon...,"[like, rear, end, slam, brake, reason, alert, ...",19,98,0,22,...,-0.339617,-0.174653,1.521694,-1.268621,-0.764868,1.950810,1,"[like, alert, ready]",3,[slam]


In [43]:
topic_dummies = []
for i in topic_model.get_topic_info()['Topic'].values:
    topic_dummies.append(f'{i}_topic')
    
topic_dummies

['-1_topic',
 '0_topic',
 '1_topic',
 '2_topic',
 '3_topic',
 '4_topic',
 '5_topic',
 '6_topic',
 '7_topic',
 '8_topic',
 '9_topic',
 '10_topic',
 '11_topic',
 '12_topic',
 '13_topic',
 '14_topic',
 '15_topic',
 '16_topic',
 '17_topic',
 '18_topic',
 '19_topic',
 '20_topic',
 '21_topic']

In [44]:
df_train[topic_dummies] = pd.get_dummies(df_train['topic'])
df_validation[topic_dummies] = pd.get_dummies(df_validation['topic'])
df_test[topic_dummies] = pd.get_dummies(df_test['topic'])
df_train.head()

,Tweet index,Label,Tweet text,emoji_count,clean_text,lemmas,word_count,char_count,all_uppercase_count,all_lowercase_count,...,12_topic,13_topic,14_topic,15_topic,16_topic,17_topic,18_topic,19_topic,20_topic,21_topic
0,2484,1,Babysitting is what I like to do on a Saturday...,1,babysitting what like saturday unamusedface,"[babysitte, like, saturday, unamusedface]",5,39,1,8,...,0,0,0,0,0,0,0,0,0,0
1,2494,0,@BigAlMLM Please give me a Follow Back.\r,0,please give follow back,[follow],4,20,0,3,...,0,0,0,0,0,0,0,0,0,0
2,1297,1,Obama wants closer relations with a Marxist na...,0,obama wants closer relations with marxist nati...,"[obama, want, close, relation, marxist, nation...",13,70,3,8,...,0,0,0,0,0,0,0,0,0,0
3,3755,1,Reading the Dundee United chairmanship stateme...,0,reading the dundee united chairmanship stateme...,"[read, dundee, united, chairmanship, statement...",18,96,0,19,...,0,0,0,0,0,0,0,0,0,0
4,3119,1,There's nothing like almost rear-ending someon...,0,there's nothing like almost rear-ending someon...,"[like, rear, end, slam, brake, reason, alert, ...",19,98,0,22,...,0,1,0,0,0,0,0,0,0,0


In [790]:
# X_ = df_train[['word_count', 'char_count', 'tag_count', 'hashtag_count', 'link_count', 'smiley_count', 'mark_count', 'has_emoji', 'neg_word_count', 'pos_word_count',
#                '-1_topic', '0_topic', '1_topic', '2_topic', '3_topic', '4_topic', '5_topic', '6_topic', '7_topic']]
# y_ = df_train['Label']
# clf = LogisticRegression(random_state=0, solver='liblinear').fit(X_, y_)

# print('Train score')
# print(clf.score(X_, y_))

# X_val = df_validation[['word_count', 'char_count', 'tag_count', 'hashtag_count', 'link_count', 'smiley_count', 'mark_count', 'has_emoji', 'neg_word_count', 'pos_word_count',
#                '-1_topic', '0_topic', '1_topic', '2_topic', '3_topic', '4_topic', '5_topic', '6_topic', '7_topic']]
# y_val = df_validation['Label']

# print('Validation score')
# print(clf.score(X_val, y_val))

# X_val = df_test[['word_count', 'char_count', 'tag_count', 'hashtag_count', 'link_count', 'smiley_count', 'mark_count', 'has_emoji', 'neg_word_count', 'pos_word_count',
#                '-1_topic', '0_topic', '1_topic', '2_topic', '3_topic', '4_topic', '5_topic', '6_topic', '7_topic']]
# y_val = df_test['Label']
# print('Test score')
# print(clf.score(X_val, y_val))

In [791]:
## Pos and neg words within 4 words
df_train[['clean_text', 'pos_word', 'pos_word_count', 'neg_word', 'neg_word_count']].iloc[25]

clean_text        whoever runs yeovil town twitter account shoul...
pos_word                                                     [like]
pos_word_count                                                    1
neg_word                                                     [fire]
neg_word_count                                                    1
Name: 25, dtype: object

In [45]:
def pos_neg_within_n(df, n=4):

    ret_array = []

    for index, row in df.iterrows():
        if row['pos_word_count'] > 0 and row['neg_word_count'] > 0:
            doc = nlp(row['clean_text'])
            lemmas = []
            for token in doc:
                lemmas.append(token.lemma_)

            pos_indexes = np.array([])
            for word in row['pos_word']:
                pos_indexes = np.append(pos_indexes, np.where(np.array(lemmas) == word))
            neg_indexes = np.array([])
            for word in row['neg_word']:
                neg_indexes = np.append(neg_indexes, np.where(np.array(lemmas) == word))

            bool_val = 0
            for idx in pos_indexes:
                if (abs(neg_indexes-idx) < n).any():
                    bool_val = 1
            ret_array.append(deepcopy(bool_val))
        else:
            ret_array.append(0)
    return ret_array

In [46]:
within_5_train = pos_neg_within_n(df_train, n=5)
within_5_val = pos_neg_within_n(df_validation, n=5)

In [47]:
df_train['pos_neg_within_5'] = within_5_train
df_validation['pos_neg_within_5'] = within_5_val

df_train.head()

,Tweet index,Label,Tweet text,emoji_count,clean_text,lemmas,word_count,char_count,all_uppercase_count,all_lowercase_count,...,13_topic,14_topic,15_topic,16_topic,17_topic,18_topic,19_topic,20_topic,21_topic,pos_neg_within_5
0,2484,1,Babysitting is what I like to do on a Saturday...,1,babysitting what like saturday unamusedface,"[babysitte, like, saturday, unamusedface]",5,39,1,8,...,0,0,0,0,0,0,0,0,0,0
1,2494,0,@BigAlMLM Please give me a Follow Back.\r,0,please give follow back,[follow],4,20,0,3,...,0,0,0,0,0,0,0,0,0,0
2,1297,1,Obama wants closer relations with a Marxist na...,0,obama wants closer relations with marxist nati...,"[obama, want, close, relation, marxist, nation...",13,70,3,8,...,0,0,0,0,0,0,0,0,0,0
3,3755,1,Reading the Dundee United chairmanship stateme...,0,reading the dundee united chairmanship stateme...,"[read, dundee, united, chairmanship, statement...",18,96,0,19,...,0,0,0,0,0,0,0,0,0,0
4,3119,1,There's nothing like almost rear-ending someon...,0,there's nothing like almost rear-ending someon...,"[like, rear, end, slam, brake, reason, alert, ...",19,98,0,22,...,1,0,0,0,0,0,0,0,0,0


In [795]:
X_ = df_train[[ 'pos_neg_within_5']]
y_ = df_train['Label']

clf = LogisticRegression().fit(X_, y_)
print('Train score')
print(clf.score(X_, y_))

X_val = df_validation[[ 'pos_neg_within_5']]
y_val = df_validation['Label']

print('Validation score')
print(clf.score(X_val, y_val))

Train score
0.5183383233532934
Validation score
0.5152838427947598


## Baseline classifiers

In [49]:
baseline_features = ['word_count', 'char_count', 'all_uppercase_count', 'all_lowercase_count', 'capitalised_count', 'digit_count']

y_train = df_train['Label']
x_train = df_train[baseline_features]

y_validation = df_validation['Label']
x_validation = df_validation[baseline_features]

y_test = df_test['Label']
x_test = df_test[baseline_features]

In [50]:
x_train.head()

,word_count,char_count,all_uppercase_count,all_lowercase_count,capitalised_count,digit_count
0,5,39,1,8,3,0
1,4,20,0,3,3,0
2,13,70,3,8,3,4
3,18,96,0,19,3,0
4,19,98,0,22,0,0


In [51]:
def model_fit_evaluate(model, x_train, y_train, x_validation, y_validation, x_test, y_test):
    model.fit(x_train, y_train)
    y_train_pred = model.predict(x_train)
    print('-------------------------- TRAIN --------------------------')
    print(classification_report(y_train, y_train_pred, digits=3), 2*'\n')
    
    y_validation_pred = model.predict(x_validation)
    print('----------------------- VALIDATION ------------------------')
    print(classification_report(y_validation, y_validation_pred, digits=3), 2*'\n')
    
    y_test_pred = model.predict(x_test)
    print('------------------------- TEST ---------------------------')
    print(classification_report(y_test, y_test_pred, digits=3), 2*'\n')

## Global baseline classifier

In [52]:
model = LogisticRegression()
model_fit_evaluate(model, x_train, y_train, x_validation, y_validation, x_test, y_test)

-------------------------- TRAIN --------------------------
              precision    recall  f1-score   support

           0      0.626     0.445     0.521      1336
           1      0.570     0.734     0.642      1336

    accuracy                          0.590      2672
   macro avg      0.598     0.590     0.581      2672
weighted avg      0.598     0.590     0.581      2672
 


----------------------- VALIDATION ------------------------
              precision    recall  f1-score   support

           0      0.635     0.453     0.529       580
           1      0.566     0.733     0.639       565

    accuracy                          0.591      1145
   macro avg      0.601     0.593     0.584      1145
weighted avg      0.601     0.591     0.583      1145
 


------------------------- TEST ---------------------------
              precision    recall  f1-score   support

           0      0.635     0.378     0.474       473
           1      0.414     0.669     0.512       31

## Global best classifier

In [53]:
pipeline_lr = Pipeline([
    ('selection', SelectKBest()), 
    ('scaler', StandardScaler()),
    ('pf', PolynomialFeatures()), 
    ('lr', LogisticRegression())
])

params = {
    'selection__k': list(range(2, 20, 3)),
    'pf__degree': [2, 3, 4, 5],
    'lr__penalty': ['l1', 'l2', 'elasticnet'],
    'lr__class_weight': ['balanced', None],
    'lr__C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 5],
    'lr__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
}


search = GridSearchCV(pipeline_lr, param_grid=params, cv=5)
search.fit(x_train, y_train)

print(f'Best score: {search.best_score_}')
print(f'Best score: {search.best_params_}')

Best score: 0.5973068710840421
Best score: {'lr__C': 0.1, 'lr__class_weight': 'balanced', 'lr__penalty': 'l1', 'lr__solver': 'liblinear', 'pf__degree': 2, 'selection__k': 5}


In [54]:
pipeline_lr_best = Pipeline([
    ('selection', SelectKBest(k=5)), 
    ('scaler', StandardScaler()),
    ('pf', PolynomialFeatures(degree=3)),
    ('lr', LogisticRegression(C=0.1, penalty='l1', solver='liblinear'))
])

model_fit_evaluate(pipeline_lr_best, x_train, y_train, x_validation, y_validation, x_test, y_test)

-------------------------- TRAIN --------------------------
              precision    recall  f1-score   support

           0      0.646     0.451     0.531      1336
           1      0.578     0.752     0.654      1336

    accuracy                          0.602      2672
   macro avg      0.612     0.602     0.593      2672
weighted avg      0.612     0.602     0.593      2672
 


----------------------- VALIDATION ------------------------
              precision    recall  f1-score   support

           0      0.600     0.421     0.494       580
           1      0.545     0.712     0.617       565

    accuracy                          0.564      1145
   macro avg      0.572     0.566     0.556      1145
weighted avg      0.572     0.564     0.555      1145
 


------------------------- TEST ---------------------------
              precision    recall  f1-score   support

           0      0.652     0.372     0.474       473
           1      0.422     0.698     0.526       31

In [809]:
pipeline_lr = Pipeline([
    #('selection', SelectKBest()), 
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression())
])

params = {
    #'selection__k': list(range(2, 20, 3)),
    'lr__penalty': ['l1'], 
    'lr__C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 2, 5],
    'lr__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
}


search = GridSearchCV(pipeline_lr, param_grid=params, cv=5)
search.fit(x_train, y_train)

print(f'Best score: {search.best_score_}')
print(f'Best score: {search.best_params_}')

Best score: 0.5916797927823865
Best score: {'lr__C': 0.1, 'lr__penalty': 'l1', 'lr__solver': 'saga'}


In [ ]:
pipeline_lr_best = Pipeline([
    ('selection', SelectKBest(k=8)), 
    ('scaler', StandardScaler()),
    ('pf', PolynomialFeatures(degree=3)),
    ('lr', LogisticRegression(C=0.1, penalty='l1', solver='liblinear'))
])

model_fit_evaluate(pipeline_lr_best, x_train, y_train, x_validation, y_validation, x_test, y_test)

## Baseline classifiers for the first 4 topics

In [803]:
def get_train_validation_and_test_for_topic(topic_num, df_train, df_validation, df_test):
    df_train_topic = df_train[df_train[f'{topic_num}_topic'] == 1]
    df_validation_topic = df_train[df_train[f'{topic_num}_topic'] == 1]
    df_test_topic = df_train[df_train[f'{topic_num}_topic'] == 1]
    
    y_train = df_train_topic['Label']
    x_train = df_train_topic[baseline_features]

    y_validation = df_validation_topic['Label']
    x_validation = df_validation_topic[baseline_features]

    y_test = df_test_topic['Label']
    x_test = df_test_topic[baseline_features]

    return x_train, y_train, x_validation, y_validation, x_test, y_test

In [804]:
x_train_, y_train_, x_validation_, y_validation_, x_test_, y_test_ = \
    get_train_validation_and_test_for_topic(-1, df_train, df_validation, df_test)

model = LogisticRegression()
model_fit_evaluate(model, x_train_, y_train_, x_validation_, y_validation_, x_test_, y_test_)

-------------------------- TRAIN --------------------------
              precision    recall  f1-score   support

           0      0.617     0.501     0.553       599
           1      0.564     0.675     0.615       573

    accuracy                          0.586      1172
   macro avg      0.591     0.588     0.584      1172
weighted avg      0.591     0.586     0.583      1172
 


----------------------- VALIDATION ------------------------
              precision    recall  f1-score   support

           0      0.617     0.501     0.553       599
           1      0.564     0.675     0.615       573

    accuracy                          0.586      1172
   macro avg      0.591     0.588     0.584      1172
weighted avg      0.591     0.586     0.583      1172
 


------------------------- TEST ---------------------------
              precision    recall  f1-score   support

           0      0.617     0.501     0.553       599
           1      0.564     0.675     0.615       57

In [805]:
x_train_, y_train_, x_validation_, y_validation_, x_test_, y_test_ = \
    get_train_validation_and_test_for_topic(0, df_train, df_validation, df_test)

model = LogisticRegression()
model_fit_evaluate(model, x_train_, y_train_, x_validation_, y_validation_, x_test_, y_test_)

-------------------------- TRAIN --------------------------
              precision    recall  f1-score   support

           0      0.632     0.542     0.583       168
           1      0.613     0.697     0.652       175

    accuracy                          0.621       343
   macro avg      0.623     0.619     0.618       343
weighted avg      0.622     0.621     0.619       343
 


----------------------- VALIDATION ------------------------
              precision    recall  f1-score   support

           0      0.632     0.542     0.583       168
           1      0.613     0.697     0.652       175

    accuracy                          0.621       343
   macro avg      0.623     0.619     0.618       343
weighted avg      0.622     0.621     0.619       343
 


------------------------- TEST ---------------------------
              precision    recall  f1-score   support

           0      0.632     0.542     0.583       168
           1      0.613     0.697     0.652       17

In [806]:
x_train_, y_train_, x_validation_, y_validation_, x_test_, y_test_ = \
    get_train_validation_and_test_for_topic(1, df_train, df_validation, df_test)

model = LogisticRegression()
model_fit_evaluate(model, x_train_, y_train_, x_validation_, y_validation_, x_test_, y_test_)

-------------------------- TRAIN --------------------------
              precision    recall  f1-score   support

           0      0.606     0.679     0.640        84
           1      0.571     0.493     0.529        73

    accuracy                          0.592       157
   macro avg      0.589     0.586     0.585       157
weighted avg      0.590     0.592     0.589       157
 


----------------------- VALIDATION ------------------------
              precision    recall  f1-score   support

           0      0.606     0.679     0.640        84
           1      0.571     0.493     0.529        73

    accuracy                          0.592       157
   macro avg      0.589     0.586     0.585       157
weighted avg      0.590     0.592     0.589       157
 


------------------------- TEST ---------------------------
              precision    recall  f1-score   support

           0      0.606     0.679     0.640        84
           1      0.571     0.493     0.529        7

In [807]:
x_train_, y_train_, x_validation_, y_validation_, x_test_, y_test_ = \
    get_train_validation_and_test_for_topic(2, df_train, df_validation, df_test)

model = LogisticRegression()
model_fit_evaluate(model, x_train_, y_train_, x_validation_, y_validation_, x_test_, y_test_)

-------------------------- TRAIN --------------------------
              precision    recall  f1-score   support

           0      0.657     0.878     0.751        74
           1      0.591     0.277     0.377        47

    accuracy                          0.645       121
   macro avg      0.624     0.577     0.564       121
weighted avg      0.631     0.645     0.606       121
 


----------------------- VALIDATION ------------------------
              precision    recall  f1-score   support

           0      0.657     0.878     0.751        74
           1      0.591     0.277     0.377        47

    accuracy                          0.645       121
   macro avg      0.624     0.577     0.564       121
weighted avg      0.631     0.645     0.606       121
 


------------------------- TEST ---------------------------
              precision    recall  f1-score   support

           0      0.657     0.878     0.751        74
           1      0.591     0.277     0.377        4

## Grid search

In [ ]:
weights = pipeline_lr_best['lr'].coef_.flatten()

plt.figure(figsize=(15, 15))
plt.bar(list(range(len(weights))), weights)
plt.ylabel('Value of the weight')
plt.xlabel('Index of the weight')
plt.title('Feature importance plot for logistic regression')
plt.show()

In [ ]:
pipeline_dtc = Pipeline([
    ('selection', SelectKBest()), 
    ('dtc', DecisionTreeClassifier()),
])

params = {
    'selection__k': list(range(2, 20, 3)),
    'dtc__max_features': ['auto', 'sqrt', 'log2'],
    'dtc__ccp_alpha': [0.1, .01, .001],
    'dtc__max_depth' : list(range(2, 10)),
    'dtc__criterion' : ['gini', 'entropy', 'log_loss']
}

search = GridSearchCV(pipeline_dtc, param_grid=params, cv=5)
search.fit(x_train, y_train)

print(f'Best score: {search.best_score_}')
print(f'Best score: {search.best_params_}')

In [ ]:
pipeline_dtc_best = Pipeline([
    ('selection', SelectKBest(k=5)), 
    ('dtc', DecisionTreeClassifier(ccp_alpha=0.001, criterion='entropy', max_depth=5, max_features='auto')),
])

model_fit_evaluate(pipeline_dtc_best, x_train, y_train, x_validation, y_validation, x_test, y_test)

In [ ]:
feat_importances = pipeline_dtc_best['dtc'].feature_importances_.flatten()

plt.figure(figsize=(5, 5))
plt.bar(list(range(len(feat_importances))), feat_importances)
plt.ylabel('Feature importance value')
plt.xlabel('Index of the feature')
plt.title('Feature importance plot for decision tree')
plt.show()